In [3]:
!pip list
!pip install pillow
!pip3 install -U scikit-learn
!pip install pandas

Package            Version
------------------ --------
asttokens          2.4.1
Brotli             1.0.9
certifi            2024.6.2
charset-normalizer 2.0.4
colorama           0.4.6
comm               0.2.2
debugpy            1.8.2
decorator          5.1.1
exceptiongroup     1.2.0
executing          2.0.1
filelock           3.13.1
gmpy2              2.1.2
idna               3.7
importlib_metadata 8.0.0
ipykernel          6.29.5
ipython            8.26.0
jedi               0.19.1
Jinja2             3.1.4
joblib             1.4.2
jupyter_client     8.6.2
jupyter_core       5.7.2
MarkupSafe         2.1.3
matplotlib-inline  0.1.7
mkl-fft            1.3.1
mkl-random         1.2.2
mkl-service        2.4.0
mpmath             1.3.0
nest_asyncio       1.6.0
networkx           3.3
numpy              1.24.3
packaging          24.1
parso              0.8.4
pickleshare        0.7.5
pillow             10.3.0
pip                24.0
platformdirs       4.2.2
prompt_toolkit     3.0.47
psutil          

In [5]:
!pip install matplotlib

  Using cached contourpy-1.2.1-cp310-cp310-win_amd64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.53.0-cp310-cp310-win_amd64.whl.metadata (165 kB)
  Using cached kiwisolver-1.4.5-cp310-cp310-win_amd64.whl.metadata (6.5 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.0 MB 660.6 kB/s eta 0:00:13
   ---------------------------------------- 0.1/8.0 MB 812.7 kB/s eta 0:00:10
    --------------------------------------- 0.1/8.0 MB 1.1 MB/s eta 0:00:08
   - -------------------------------------- 0.3/8.0 MB 2.3 MB/s eta 0:00:04
   - -------------------------------------- 0.4/8.0 MB 2.2 MB/s eta 0:00:04
   ---- ----------------------------------- 0.9/8.0 MB 4.4 MB/s eta 0:00:02
   ----- ---------------------------------- 1.1/8.0 MB 4.3 MB/s eta 0:00:02
   ------- ---------------------------

In [6]:
import torch
import torchvision
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


In [10]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
import pandas as pd
from torchvision import transforms

class CoconutDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")

        bbox_string = self.annotations.iloc[idx, 1]
        bbox_list = list(map(int, bbox_string.split()))
        boxes = []

        # Group coordinates into (y_min, x_min, y_max, x_max)
        for i in range(0, len(bbox_list), 4):
            y_min = bbox_list[i]
            x_min = bbox_list[i + 1]
            y_max = bbox_list[i + 2]
            x_max = bbox_list[i + 3]
            boxes.append([x_min, y_min, x_max, y_max])  # Convert to (x_min, y_min, x_max, y_max)

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)  # Assuming all objects are coconuts (label 1)

        target = {"boxes": boxes, "labels": labels}

        if self.transform:
            image = self.transform(image)

        return image, target


In [11]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

In [18]:
current_dir = os.getcwd()
csv_file = 'D:\\code\\CoconutCount\\data\\train\\mbb_noempties.csv'
image_dir = 'D:\\code\\CoconutCount\\data\\train\\jpegs'


dataset = CoconutDataset(csv_file=csv_file, root_dir=image_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
